In [1]:
import mwclient
import time
import transformers

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Ethereum']

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1226381272),
             ('parentid', 1225878722),
             ('user', 'Hatrick24'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=5, tm_mday=30, tm_hour=8, tm_min=4, tm_sec=46, tm_wday=3, tm_yday=151, tm_isdst=-1)),
             ('comment', 'Updated')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [5]:
revs[0]

OrderedDict([('revid', 592567939),
             ('parentid', 0),
             ('user', 'Sanpitch'),
             ('timestamp',
              time.struct_time(tm_year=2014, tm_mon=1, tm_mday=27, tm_hour=1, tm_min=53, tm_sec=45, tm_wday=0, tm_yday=27, tm_isdst=-1)),
             ('comment',
              "[[WP:AES|←]]Created page with '{{Infobox currency | image_1 =  | image_title_1 =  | image_width_1 =  | image_2 =  | image_title_2 =  | image_width_2 =  |issuing_authority = None. The Ethereum...'")])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\SUBRAT\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to forc

In [7]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [8]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [9]:
import pandas as pd
edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [10]:
edits_df

,edit_count,sentiment,neg_sentiment
2014-01-27,1,-0.998511,1.000000
2014-02-01,1,-0.997276,1.000000
2014-04-06,5,0.790980,0.000000
2014-04-09,24,0.646407,0.083333
2014-04-10,9,-0.361518,0.666667
...,...,...,...
2024-05-24,1,-0.995769,1.000000
2024-05-25,4,0.482980,0.250000
2024-05-26,1,0.975446,0.000000
2024-05-27,2,-0.011985,0.500000


In [11]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits

,edit_count,sentiment,neg_sentiment
2014-09-08,3.633333,0.219135,0.355397
2014-09-30,3.633333,0.285044,0.322063
2014-10-08,3.666667,0.347025,0.288730
2014-10-14,3.566667,0.320337,0.305397
2014-10-23,2.900000,0.298716,0.319286
...,...,...,...
2024-05-24,1.600000,-0.340325,0.664444
2024-05-25,1.700000,-0.291719,0.639444
2024-05-26,1.700000,-0.234965,0.606111
2024-05-27,1.666667,-0.265106,0.622778


In [12]:
rolling_edits.to_csv("ethereum_sentiment.csv")